# When to Use Object-Oriented Programming

### Topics
- how to recognize objects
- data and behaviors, once again
- wrapping data behaviors using properties
- DRY principle and avoiding repetition

## Treat objects as objects

- identify objects in the problem, and then model their data and behaviors
- if data is the focus of a problem, use Python data structures such as list, dictionary, set, etc.
- if behavior is the focus, simple function is more suitable
- objects have both data and behavior and are of equal focus
- e.g., design a program to model polygons in two-dimensional space
    - we could start with each polygon represented as a list of points - list of tuples
    - then convert it into OOD
- OOD can be verbose; lot more lines of code compared to the functional counterpart
- Code length, however, is not a good indicator of code complexity
- *one-liner* can be a fun exercise but the result is often unreadable, even to the original author the next day
- the more important a set of data is, the more likely it is to have multiple functions specfic to that data
    - more useful to use a class with attributes and methods instead
- certain interactions among objects, and inheritance can't be modeled elegantly without classes
- composition can be modeled with data structures only, but the code is less readable
    - e.g., we can have list of dictionaries holding tuple values
    - less obvious what those tuples or dictionaries hold/represent!
    
#### NOTE: No one wins at code golf. Minimizing the volume of code is rarely desirable.

In [1]:
# use built-in data structure to represent a square polygon
square = [(1, 1), (1, 2), (2, 2), (2, 1)]

In [7]:
from math import hypot

# now find the perimeter of the square
def distance(p1, p2):
    return hypot(p1[0]-p2[0], p1[1]-p2[1])

In [8]:
def perimeter(polygon):
    # zip creates: (v[0], v[1]), (v[1], v[2]), and (v[2], v[0])
    pairs = zip(polygon, polygon[1:]+polygon[:1])
    return sum(distance(p1, p2) for p1, p2 in pairs)

In [9]:
perimeter(square)

4.0

In [11]:
from math import hypot
from typing import Tuple, List

# Type alias
Point = Tuple[float, float] # this can be converted into a class

def distance(p_1: Point, p_2: Point) -> float:
    return hypot(p_1[0] - p_2[0], p_1[1] - p_2[1])

Polygon = List[Point] # this can be converted into a class

def perimeter(polygon: Polygon) -> float:
    pairs = zip(polygon, polygon[1:] + polygon[:1])
    return sum(distance(p1, p2) for p1, p2 in pairs)

In [13]:
from math import hypot
from typing import Tuple, List, Optional, Iterable

class Point:
    def __init__(self, x: float, y: float) -> None:
        self.x = x
        self.y = y
        
    def distance(self, other: "Point") -> float:
        return hypot(self.x - other.x, self.y - other.y)
    
class Polygon:
    def __init__(self) -> None:
        self._vertices: List[Point] = []
            
    def add_point(self, point: "Point") -> None:
        self._vertices.append((point))
        
    def perimeter(self) -> float:
        pairs = zip(self._vertices, self._vertices[1:] + self._vertices[:1])
        return sum(p1.distance(p2) for p1, p2 in pairs)

In [14]:
square = Polygon()

In [15]:
square.add_point(Point(1, 1))

In [16]:
square.add_point(Point(1, 2))

In [17]:
square.add_point(Point(2, 2))

In [18]:
square.add_point(Point(2, 1))

In [19]:
square.perimeter()

4.0

In [20]:
# more verbose/steps, compared to functional design
square = [(1, 1), (1, 2), (2, 2), (2, 1)]

In [21]:
perimeter(square)

4.0

In [22]:
# we could shorten Polygon class

class Polygon_2:
    def __init__(self, vertices: Optional[Iterable[Point]] = None) -> None:
        self._vertices = list(vertices) if vertices else []
        
    def perimeter(self) -> float:
        pairs = zip(self._vertices, self._vertices[1:]+self._vertices[:1])
        return sum(p1.distance(p2) for p1, p2 in pairs)

In [23]:
square = Polygon_2([Point(1, 1), Point(1, 2), Point(2, 2), Point(2, 1)])

In [24]:
square.perimeter()

4.0

In [36]:
# let's take one more step; allow Polygon to accet tuples too
from typing import Union

Pair = Tuple[float, float]
Point_or_Tuple = Union[Point, Pair]

class Polygon_3:
    def __init__(self, vertices: Optional[Iterable[Point_or_Tuple]] = None) -> None:
        self._vertices: List[Point] = []
        if vertices:
            for point_or_tuple in vertices:
                self._vertices.append(Polygon_3.make_point(point_or_tuple))
                
    def perimeter(self) -> float:
        pairs = zip(self._vertices, self._vertices[1:]+self._vertices[:1])
        return sum(p1.distance(p2) for p1, p2 in pairs)
    
    @staticmethod
    def make_point(item: Point_or_Tuple) -> Point:
        return item if isinstance(item, Point) else Point(*item)
    

In [37]:
square = Polygon_3([(1, 1), (1, 2), (2, 2), (2, 1)])

In [38]:
square.perimeter()

4.0

## Adding behaviors to class data with properties

- eventhough classes hold both data and behavior, we've explictly separated them with variables and methods
- the distinction between data and behavior sometimes can be uncannily blurry
- OOD teaches us to never access attributes directly
    - OO deverlopers insist we write setters and getters (methods) to access attributes
- setter and getter methods are not as readable and is not favored in my OOP languages
    - they provide a simpler syntax to access attribues "as if" you're directly accessing them without the methods
- why setters and getters?
    - comiple code cleanly as functions
    - write extra code for data validation, caching (avoid complex recomputation), etc. when accessing and setting
- Python gives us `property` function that make methods *look* like attributes
- or use **decorators** `@property`, `@method.setter`, `@method.deleter` to define properties

In [39]:
class Color:
    def __init__(self, rgb_value: int, name: str) -> None:
        self._rgb_value = rgb_value
        self._name = name
        
    def set_name(self, name: str) -> None:
        self._name = name
        
    def get_name(self) -> str:
        return self._name
    
    def set_rgb_value(self, rgb_value: int) -> None:
        self._rgb_value = rgb_value
        
    def get_rgb_value(self) -> int:
        return self._rgb_value

In [40]:
c = Color(0xff0000, "bright red")

In [41]:
c.get_name()

'bright red'

In [42]:
c.set_name("Red")

In [43]:
c.get_name()

'Red'

In [44]:
# favored syntax

class Color_Py:
    def __init__(self, rgb_value: int, name: str) -> None:
        self.rgb_value = rgb_value
        self.name = name

In [47]:
c = Color_Py(0xff0000, "bright red")

In [48]:
c.name

'bright red'

In [49]:
c.name = 'Red'

In [50]:
c.name

'Red'

In [54]:
# let's use property function

class Color_VP:
    
    def __init__(self, rgb_value: int, name: str) -> None:
        self._rgb_value = rgb_value
        # redundant code!
        if not name:
            raise ValueError(f"Invalid name {name!r}")
        self._name = name
        
    def _set_name(self, name: str) -> None:
        if not name:
            raise ValueError(f"Invalid name {name!r}")
        self._name = name
        
    def _get_name(self) -> str:
        return self._name
    
    # must use property after the getter and setters are defined
    name = property(_get_name, _set_name)


In [55]:
c = Color_VP(0xff0000, "bright red")

In [56]:
c.name

'bright red'

In [57]:
c.name = "Red"

In [58]:
# should throw an ValueError excpetion
c.name = ''

ValueError: Invalid name ''

### Properties in detail

- `property` constructor can actually accept two additional arguments
    - a `delete` function: delete associated attribute if necessary
    - a `docstring` for the property


In [68]:
class NorwegianBlue:
    def __init__(self, name: str) -> None:
        self._name = name
        self._state: str
    
    def _get_state(self) -> str:
        print(f"Getting {self._name}'s state")
        return self._state
    
    def _set_state(self, state: str) -> str:
        print(f"Setting {self._name}'s State to {state!r}")
        self._state = state
        
    def _del_state(self) -> None:
        print(f"{self._name} is pushing up daisies!")
        del self._state
        
    silly = property(_get_state, _set_state, _del_state,
                    "This is a silly property")

In [69]:
p = NorwegianBlue("Polly")

In [70]:
p.silly = "Pinning for the fjords"

Setting Polly's State to 'Pinning for the fjords'


In [71]:
p.silly

Getting Polly's state


'Pinning for the fjords'

In [72]:
del p.silly

Polly is pushing up daisies!


In [73]:
# see the docstring of silly
help(p)

Help on NorwegianBlue in module __main__ object:

class NorwegianBlue(builtins.object)
 |  NorwegianBlue(name: str) -> None
 |  
 |  Methods defined here:
 |  
 |  __init__(self, name: str) -> None
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  ----------------------------------------------------------------------
 |  Data descriptors defined here:
 |  
 |  __dict__
 |      dictionary for instance variables (if defined)
 |  
 |  __weakref__
 |      list of weak references to the object (if defined)
 |  
 |  silly
 |      This is a silly property



In [76]:
# using decorator
class NorwegianBlue_P:
    def __init__(self, name: str) -> None:
        self._name = name
        self._state: str
            
    @property
    def silly(self) -> str:
        """This is a silly property"""
        print(f"Getting {self._name}'s State")
        return self._state
    
    @silly.setter
    def silly(self, state: str) -> None:
        print(f"Setting {self._name}'s State to {state!r}")
        self._state = state
        
    @silly.deleter
    def silly(self) -> None:
        print(f"{self._name} is pushing up daisies!")
        del self._state

## When to use properties

- can be confusing when properties blur the distinction between behavior and data
- properties, data, behavior are generally called the attributes of a class
- follow these principles:
    - use methods to represent actions; method names are generally verbs
    - use attributes or properties to represent the state of the object; these are nouns, adjectives, and prepositions
        - use properties for attributes in the exceptional case when:
            - complex computation is involved such as data validation, logging, access controls, caching, etc.
- example of caching data

In [77]:
from urllib.request import urlopen
from typing import Optional, cast

class WebPage:
    def __init__(self, url: str) -> None:
        self.url = url
        self._content: Optional[bytes] = None
            
    @property
    def content(self) -> bytes:
        if self._content is None:
            print("Retrieving New Page...")
            with urlopen(self.url) as response:
                self._content = response.read()
        return self._content

In [78]:
import time
webpage = WebPage("http://ccphillips.net/")
now = time.perf_counter()
content1 = webpage.content
first_fetch = time.perf_counter() - now

first_fetch = time.perf_counter() - now
now = time.perf_counter()
content2 = webpage.content
second_fetch = time.perf_counter() - now
assert content2 == content1, "Problem: Pages were different"
print(f"Initial Request     {first_fetch:.5f}")
print(f"Subsequent Requests {second_fetch:.5f}")

Retrieving New Page...
Initial Request     4.52186
Subsequent Requests 0.00008


## Manager objects

- higher level objects that manage other objects
    - the object that ties everything together
- these are sometimes called Facade objects because they present a pleasant, easy-to-use facade over some underlying complexity
    - Facade is also a design pattern that's covered in more detail later in the course
- similar to managers in officer, management object, may not do the actual work
- the attributes on a management class tend to refer to other objects that do the visible work;
    - managers typically delegate to other classes at the right time, and pass messages between them
- assemble manager objects by knitting other objects together
- to an extent, a manager is also an Adapter among the various interfaces
    - another design pattern covered later in the course
- Solution/Main class in `python3/OOP` subfolders of Kattis demos (https://github.com/rambasnet/Kattis-Demos-Testing) is example of manager class

## Removing duplicate code

- Why is duplicate code bad thing?
    - boils down to 2 reasons: **readability** and **maintainability**
    
- **copy-pasta** programming: copy/pasting similar code to extend/add new functionalities
    - creates a big mess of tangled noodles of code like a bown of spaghetti
- when we see similar or near duplicate code blocks, we have *additional* intellectual barrier to understanding:
    1. Are they truly identical?
    2. If not, how is one section different from the other?
    3. When do we call the other?
- you may be the only person working on the project and it may all makes sense to you now
    - just easy to copy and paste! However, can you remember the logic behind it next day/next year? What if someone else takes over/joins the project?

#### NOTE: Code should always be written to be readable first.

### DRY principle

- Python developers prefer elegant, clean code and follow the **Don't Repeat Yourself** principle
- Never copy and paste code! Think thrice before hitting `Ctrl+C`